In [ ]:
import pandas as pd
import re

from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Mecab
from konlpy.tag import Okt

from collections import Counter
from itertools import chain

In [ ]:
# DEPRECATED: KoNLPy에서 추출된 한글 명사 리스트에서 2글자 미만으로 구성된 단어 삭제
#
# 1. Sample Data/Input Example
# input_data = [['사과', '가', '어디']]
# output_data = remove_below_two_char_in_noun_voca_list(input_data)
#
# 2. Output Example/Data
# output_data: [['사과, '어디']]

def remove_below_two_char_in_noun_voca_list(noun_voca_list):
    changed_noun_voca_list = []
    for noun_voca_line in noun_voca_list:
        changed_noun_voca_line = [noun_word for noun_word in noun_voca_line if len(noun_word) > 1]
        changed_noun_voca_list.append(changed_noun_voca_line)
    return changed_noun_voca_list

In [ ]:
# DEPRECATED: KoNLPy에서 추출된 한글 명사 리스트에서 특정 단어 삭제
# 
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['딸기', '산딸기']]
# output_data = remove_text_from_noun_voca_list(input_data, '딸기')
# 
# 2. Output Example/Data
# 해당 글자가 포함된 단어가 아래와 같이 검출되었음
# 산딸기
# output_data: [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['산딸기']]

def remove_text_from_noun_voca_list(noun_voca_list, text):
    similar_noun_voca_list = []
    changed_noun_voca_list = []
    for noun_voca_line in noun_voca_list:
        for noun_word in noun_voca_line:
            if text in noun_word:
                if noun_word in similar_noun_voca_list:
                    pass
                elif text != noun_word:
                    similar_noun_voca_list.append(noun_word) 
            if text is noun_voca_line:
                pass
            else:
                changed_noun_voca_list.append(noun_voca_line)
    if len(similar_noun_voca_list)==0:
        pass
    else:
        print("해당 글자가 포함된 단어가 아래와 같이 검출되었음")
        print(", ".join(similar_noun_voca_list))
    return changed_noun_voca_list

In [ ]:
# # DEPRECATED: KoNLPy에서 추출된 한글 명사 리스트에서 두 글자로 이루어진 단어만 추출 후 STOPWORDS 삭제
#
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['딸기', '산딸기']]
# stopwords = ['딸기']
# output_data = remove_stopwords(input_data, stopwords)
#
# 2. Output Example/Data
# output_data: [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['산딸기']]

def remove_stopwords(voca_list, stopword_list):
    stopword_replaced_voca_list = []
    sentence_list = make_sentence_list_from_voca_list(voca_list)
    re_space = re.compile(r'\s+')

    for line in sentence_list:
        for stopword in stopword_list:
            line = ' '.join(re.compile('[가-힣]{2,}').findall(line))
            re_word = re.compile(r'\b{0}\b'.format(stopword))
            line = (re_space.sub(' ', re_word.sub('', line)).strip())
        stopword_replaced_voca_list.append(line.split(' '))

    return stopword_replaced_voca_list

In [ ]:
# Konlpy를 이용하여 명사 추출
#
# 1. Sample Data/Input Exmaple
# input_data = ['구아바 구아바 망고를 유혹하네 딱 걸렸네, 포시즌을 낳았네', '젖소방은 없습니다.']
# konlpy_opt = 'Hannanum', 'Kkma', 'Komoran', 'Mecab', 'Okt' 중 1개 입력
# Konlpy tag Package: https://konlpy-ko.readthedocs.io/ko/latest/api/konlpy.tag/
# input: output_data = make_noun_voca_list(input_data, konlpy_opt)
#
# 2. Output Example/Data
# output_data: [['구아바', '구아바', '망고', '유혹', '포', '시즌'], ['젖', '소방']]

def make_noun_voca_list(raw_list, konlpy_opt):
    noun_voca_list = []
    assert konlpy_opt in ['Hannanum', 'Kkma', 'Komoran', 'Mecab', 'Okt']

    if konlpy_opt == 'Hannanum':
        nlp = Hannanum()
    elif konlpy_opt == 'Kkma':
        nlp = Kkma()
    elif konlpy_opt == 'Komoran':
        nlp = Komoran()
    elif konlpy_opt == 'Mecab':
        nlp = Mecab()
    elif konlpy_opt == 'Okt':
        nlp = Okt()

    for line in raw_list:
        if type(line).__name__ == 'float' or type(line).__name__ == 'int':
            noun_voca_list.append('')
        else:
            noun_voca_list.append(nlp.nouns(' '.join(re.compile('[가-힣0-9]+').findall(line))))

    return noun_voca_list

In [ ]:
# 단어 리스트를 띄어쓰기를 기준으로 join
#
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바']]
# input: output_data = make_sentence_list_from_voca_list(input_data)
#
# 2. Output Example/Data
# output_data: ['사과 바나나 포도', '수박 파인애플 구아바']

def make_sentence_list_from_voca_list(voca_list):
    sentence_list = []
    for voca_line in voca_list:
        sentence_list.append(" ".join(voca_line))
    return sentence_list

In [ ]:
# 문장을 띄어쓰기를 기준으로 분할하여 list화
#
# 1. Sample Data/Input Exmaple
# input_data = ['사과 바나나 포도', '수박 파인애플 구아바']
# output_data = make_voca_list_from_sentence_list(input_data)
#
# 2. Output Example/Data
# output_data: [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바']]

def make_voca_list_from_sentence_list(sentence_list):
    voca_list = []
    for sentence_line in sentence_list:
        voca_list.append(sentence_line.split(' '))
    return voca_list

In [ ]:
# KoNLPy에서 추출된 한글 명사 리스트에서 두 글자로 이루어진 단어만 추출 후 STOPWORDS 삭제
#
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['딸기', '산딸기']]
# stopwords = ['딸기']
# output_data = remove_stopwords(input_data, stopwords)
#
# 2. Output Example/Data
# output_data: [['사과', '바나나', '포도'], ['수박', '파인애플', '구아바'], ['산딸기']]

def remove_stopwords(voca_list, stopword_list):
    stopword_replaced_voca_list = []

    for line in voca_list:
        stopword_replaced_voca_list.append(list(filter(lambda x: len(x) > 1 and x not in stopword_list, line)))

    return stopword_replaced_voca_list

In [ ]:
# 단어 빈도순 출력
#
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '사과'], ['딸기', '사과', '바나나', '수박']]
# top_count = n (int, 상위 n위)
# print_most_common_words(input_data, 4)
#
# 2. Output Example/Data
# output_data: [('사과', 3), ('바나나', 2), ('딸기', 1), ('수박', 1)]
# 주의: 동 순위가 있다면 앞에 있는 element 순서대로 끊어짐 (예: [('사과', 3), ('바나나', 2), ('딸기', 1)])

def print_most_common_words(voca_list, top_count):
    print(Counter(list(chain.from_iterable(voca_list))).most_common(top_count))

In [ ]:
# 단어 빈도 데이터 저장
#
# 1. Sample Data/Input Exmaple
# input_data = [['사과', '바나나', '사과'], ['딸기', '사과', '바나나', '수박']]
# top_count = n (int, 상위 n위, 저장 CSV 파일 경로)
# print_most_common_words(input_data, 4)
#
# 2. Output Example/Data
# output_data: CSV 파일
# Column Name: WORD, COUNT
# 주의: 동 순위가 있다면 앞에 있는 element 순서대로 끊어짐 (예: [('사과', 3), ('바나나', 2), ('딸기', 1)])

def save_most_common_words(voca_list, top_count, output_path):
    word_count_list = Counter(list(chain.from_iterable(voca_list))).most_common(top_count)
    word_list = []
    count_list = []
    for idx in range(len(word_count_list)):
        word_list.append(word_count_list[idx][0])
        count_list.append(word_count_list[idx][1])
    word_count_df = pd.DataFrame.from_records(zip(word_list, count_list), columns=['WORD', 'COUNT'])

    word_count_df.to_csv(output_path, sep=',', index=False, encoding='utf-8-sig')
    

In [ ]:
raw_path = 'data/sample_newspaper.csv'
raw_list_name = '기사'

raw_df = pd.read_csv(raw_path, sep=',', encoding='utf-8')
raw_list = raw_df[raw_list_name]

In [ ]:
raw_noun_voca_list = make_noun_voca_list(raw_list, 'Okt')

In [ ]:
stopwords = pd.ExcelFile('stopwords_ko_20191105.xlsx').parse('STOPWORDS').STOPWORDS
stopword_replaced_voca_list = remove_stopwords(raw_noun_voca_list, stopwords)

In [ ]:
stopwords_custom = ['우리', '최근']
stopword_replaced_voca_list = remove_stopwords(stopword_replaced_voca_list, stopwords_custom)

In [ ]:
print_most_common_words(stopword_replaced_voca_list, 200)

In [ ]:
common_words_path = 'data/sample_newspaper_common_word.csv'
save_most_common_words(stopword_replaced_voca_list, 200, common_words_path)

In [ ]:
raw_noun_sentence_list = make_sentence_list_from_voca_list(stopword_replaced_voca_list)

In [ ]:
output_path = 'data/sample_newspaper_modified.csv'
output_list_name = '기사_명사'

raw_df[output_list_name] = raw_noun_sentence_list
raw_df.to_csv(output_path, sep=',', index=False, encoding='utf-8-sig')